Кирилл Лалаянц, R33352

# Лабораторная работа No7
# В -- 11
## Управляемость и наблюдаемость

Импорт необходимых для работы библиотек. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import control 
import sympy
import os
import scipy

SAVE_PATH = 'tex-report/src/figs/'
os.makedirs(SAVE_PATH, exist_ok=True)

sympy.init_printing()
p = sympy.Symbol("p")
s = sympy.Symbol("s")
t = sympy.Symbol("t")
w = sympy.Symbol("w")
I = sympy.I


def get_t(end_t = 10, dt=0.001, start_t = 0):
    return np.linspace(start_t, end_t, int(end_t / dt))

## Task1

In [ ]:
task1_A = np.array([
    [3, 4, -1],
    [-10, -11, -4],
    [10, 10, 3]
])

task1_B = np.array([[-2], [5], [-3]])

task1_x1 = np.array([[-2], [1], [-1]])
task1_end_time = 3

In [ ]:
def get_controllability_matrix(A, B):
    ctrb_m = np.hstack((B, *[(np.linalg.matrix_power(A, i)) @ B for i in range(1, A.shape[0])]))
    assert np.allclose(control.ctrb(A, B), ctrb_m), 'Smth wrong'
    return ctrb_m

def check_controllability(A, B, end_time):
    print('======== System check ==========')
    A_sympy, B_sympy = sympy.Matrix(A), sympy.Matrix(B)
    U = get_controllability_matrix(A, B)
    print(f'U: \n{U}')
    print(f'RankU = {np.linalg.matrix_rank(U)}')
    print(f'Eigen values of A: {np.linalg.eigvals(A)}')
    A_P, A_J = A_sympy.jordan_form()
    print(f'Jordan of A: {A_J}')
    print(f'B^ = {(A_P **-1 ) * B_sympy}')
    control_gramian_matrix, control_gramian_error = get_controllability_gramian(A, B, end_time)
    print(f'Controllability Gramian at {end_time}s: \n{control_gramian_matrix}')
    print(f'Eigen values of Controllability Gramian at {end_time}s: {np.linalg.eigvals(control_gramian_matrix)}')
    

def check_controllability_of_point(A, B, x1):
    print(f'======== Controllability of {x1.reshape(-1)} ==========')
    U = get_controllability_matrix(A, B)
    U_extended = np.hstack((U, x1))
    rgU_extended = np.linalg.matrix_rank(np.hstack((U, x1)))
    print(f'U_ext: \n {U_extended}')
    # print(np.linalg.svd(U_extended, compute_uv=False))
    print(f'Rang(U_ext) = {rgU_extended}')
    controllable =  rgU_extended == A.shape[0]
    print(f'In Controllable Space: {controllable}')
    return controllable

def get_controllability_gramian(A, B, end_time):
    f = lambda time: scipy.linalg.expm(A * time) @ B @ B.T @ scipy.linalg.expm(A.T * time)
    return scipy.integrate.quad_vec(f, 0, end_time)

def get_u_for_controllable(A, B, x1, end_time):
    u = []
    times = get_t(end_time)
    control_gramian_matrix, _ = get_controllability_gramian(A, B, end_time)
    for t in times:
        u.append(B.T @ scipy.linalg.expm(A.T * (end_time - t)) @ np.linalg.inv(control_gramian_matrix) @ x1)
    return np.array(u).reshape((B.shape[1], -1)), times

def plot_task1(A, B, x1, end_time, file_name='test.jpg'):
    u, times = get_u_for_controllable(A, B, x1, end_time)
    ss = control.ss(A, B, A * 0, B * 0)
    output = control.forced_response(ss, X0=[0,0,0], U=u, T=times).states

    fig, axs = plt.subplots(2, 2, figsize=(8, 8))
    plt.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, wspace=0.4,hspace=0.4)
    for i in range(3):
        axs[i//2, i % 2].plot(times, output[i], linewidth=4)
        axs[i//2, i % 2].scatter(end_time, x1[i], color='r', linewidth=4, label='goal')
        axs[i//2, i % 2].set_title(f'x_{i} = x_{i}(t)', fontsize=12)
        axs[i//2, i % 2].set_xlabel(f"t, [c]", fontsize=12)
        axs[i//2, i % 2].grid(True)
        axs[i//2, i % 2].legend()
        
        
    axs[1, 1].plot(times, u.reshape(-1), linewidth=4)
    axs[1, 1].set_title(f'u = u(t)', fontsize=12)
    axs[1, 1].set_xlabel(f"t, [c]", fontsize=12)
    axs[1, 1].grid(True)
    plt.savefig(f'{SAVE_PATH}/{file_name}.jpg') 

In [ ]:
check_controllability(task1_A, task1_B, task1_end_time)
check_controllability_of_point(task1_A, task1_B, task1_x1)

In [ ]:
plot_task1(task1_A, task1_B, task1_x1, task1_end_time, 'task1')

## Task 2

In [ ]:
task2_A = np.array([
    [3, 4, -1],
    [-10, -11, -4],
    [10, 10, 3]
])

task2_B = np.array([[2], [1], [-1]])

task2_mbx1 = np.array([[-2], [1], [-1]])
task2_mbx2 = np.array([[-5], [4], [-1]])

task2_end_time = 3

In [ ]:
check_controllability(task2_A, task2_B, task2_end_time)
is_mbx1 = check_controllability_of_point(task2_A, task2_B, task2_mbx1)
is_mbx2 = check_controllability_of_point(task2_A, task2_B, task2_mbx2)

assert is_mbx1 + is_mbx2 == 1, 'Two or zero controllable points'
task2_x1 = task2_mbx1 if is_mbx1 else task2_mbx2

In [ ]:
plot_task1(task2_A, task2_B, task2_x1, task2_end_time, 'task2')